## This notebook contains the process of data acquisition from three data sources:
1. American Community Survey (ACS) 5-Year Data (2009-2018)

2. AARP.org Livability Index

3. Census Mapping Files: TIGER/Line Shapefiles

In [21]:
import io
import requests
import json
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd

from pandas import DataFrame
from shapely.geometry import Point
from IPython.display import GeoJSON

# Read median gross rent from US Census data

In [2]:
API_KEY = '8e8e73ee55d6949e46bd4f555c948d27a1337aa4'

In [3]:
def get_census_data(variable, state_id, county_id):
    with requests.session() as session:
        params = {
            'get': variable,
            'for': 'block group:*',
            'in': f'state:{state_id} county:{county_id}',
            'key': API_KEY
        }
        url = f'https://api.census.gov/data/2018/acs/acs5'
        response = session.get(url, params=params)
        try:
            response.raise_for_status()
        except Exception as e:
            raise Exception(f'Failed to query {url} due to: {response.text}') from e
        result = response.json()
        df = DataFrame(result[1:], columns=result[0])
    return df

get_census_data('B25058_001E', '34', '001')

,B25058_001E,state,county,tract,block group
0,1188,34,001,012402,2
1,985,34,001,010900,3
2,1365,34,001,011500,2
3,-666666666,34,001,011500,3
4,909,34,001,012801,1
...,...,...,...,...,...
179,-666666666,34,001,013301,2
180,1128,34,001,013302,2
181,1632,34,001,013500,2
182,-666666666,34,001,990000,0


# Read AARP livabiilty data

In [16]:
def merge_properties_and_demographics(livability_data):  # geopandas cannot regnoize 'demographics' but only 'properties', so move 'demo' to 'prop'
    for f in livability_data['features']:
        dem = f.pop('demographics')
        f['properties'].update(dem)
    return livability_data

# Single call to AARP without handling if the result size is larger than the 1000-block-group limit
def read_aarp_livability(swx, swy, nex, ney):
    url = f'https://geoapp.livabilityindex.byf1.io/demographicGeo?swX={swx}&swY={swy}&neX={nex}&neY={ney}&geo_type=blockgroup&simplificationLevel=0.0001'
    print(url)
    with requests.session() as session:
        response = session.get(url)
        try:
            response.raise_for_status()
        except Exception as e:
            if response.text == 'No rows returned for query.':
                return None 
            raise Exception(f'Failed to query {url} due to: {response.text}') from e
        raw_data = response.json()
        # merge the demographics fields from `raw_data` into `properties` fields such that GeoPandas could 
        # recognize and put them in the result dataframe
        merged_data = merge_properties_and_demographics(raw_data)
        # use GeoPandas to read the data, use `io.StringIO()` to convert a string to a in-memory file
        df = gpd.read_file(io.StringIO(json.dumps(merged_data)))
    return df

In [17]:
aa = read_aarp_livability(-74.04883861541748, 40.69866554506028, -74.01626586914062, 40.72810434130303)
aa.head()

https://geoapp.livabilityindex.byf1.io/demographicGeo?swX=-74.04883861541748&swY=40.69866554506028&neX=-74.01626586914062&neY=40.72810434130303&geo_type=blockgroup&simplificationLevel=0.0001


,state_fips,total_popu,pct_africa,pct_asian,pct_hispan,pct_50plus,pct_65plus,median_inc,pct_povert,pct_disabi,...,prox_vacant,trans_freq,trans_access,trans_walk_trip,trans_delay,trans_cost,trans_fatal,trans_speed_lim,total_score,geometry
0,34,1931,7,11,15,31,9,174722,0,9,...,0.106,0.0,0.933,0.59,32.996,11689,3.052,31.86,47.428571,"MULTIPOLYGON (((-74.09860 40.68228, -74.09636 ..."
1,34,0,0,0,0,0,0,0,0,9,...,0.116,0.0,0.933,0.52,32.996,13820,2.658,27.27,44.571429,"MULTIPOLYGON (((-74.06875 40.70099, -74.06722 ..."
2,36,0,0,0,0,0,0,59190,0,10,...,0.116,0.0,0.933,0.48,32.996,13820,2.627,25.00,45.428571,"MULTIPOLYGON (((-74.04075 40.70017, -74.04008 ..."
3,36,0,0,0,0,0,0,0,0,10,...,0.116,0.0,0.933,0.49,32.996,13820,2.494,26.03,41.000000,"MULTIPOLYGON (((-74.03294 40.68778, -74.02628 ..."
4,36,1754,3,21,12,5,0,139219,12,10,...,0.263,739.0,0.933,5.55,32.996,7622,2.650,28.04,54.571429,"MULTIPOLYGON (((-74.01661 40.70496, -74.01551 ..."


In [18]:
# GeoJSON(json.loads(aa.to_json()))

# Read shapefiles / block groups

## State IDs

The state id data was scaped from https://www.census.gov/cgi-bin/geo/shapefiles/index.php

In [23]:
state_id_filename = 'data_aarp_census/census/block_group_shapefiles/shapefile_state_ids.json'
state_ids = json.loads(Path(state_id_filename).read_text())
state_ids

{'01': 'Alabama',
 '02': 'Alaska',
 '60': 'American Samoa',
 '04': 'Arizona',
 '05': 'Arkansas',
 '06': 'California',
 '08': 'Colorado',
 '69': 'Commonwealth of the Northern Mariana Islands',
 '09': 'Connecticut',
 '10': 'Delaware',
 '11': 'District of Columbia',
 '12': 'Florida',
 '13': 'Georgia',
 '66': 'Guam',
 '15': 'Hawaii',
 '16': 'Idaho',
 '17': 'Illinois',
 '18': 'Indiana',
 '19': 'Iowa',
 '20': 'Kansas',
 '21': 'Kentucky',
 '22': 'Louisiana',
 '23': 'Maine',
 '24': 'Maryland',
 '25': 'Massachusetts',
 '26': 'Michigan',
 '27': 'Minnesota',
 '28': 'Mississippi',
 '29': 'Missouri',
 '30': 'Montana',
 '31': 'Nebraska',
 '32': 'Nevada',
 '33': 'New Hampshire',
 '34': 'New Jersey',
 '35': 'New Mexico',
 '36': 'New York',
 '37': 'North Carolina',
 '38': 'North Dakota',
 '39': 'Ohio',
 '40': 'Oklahoma',
 '41': 'Oregon',
 '42': 'Pennsylvania',
 '72': 'Puerto Rico',
 '44': 'Rhode Island',
 '45': 'South Carolina',
 '46': 'South Dakota',
 '47': 'Tennessee',
 '48': 'Texas',
 '49': 'Utah',


## State-County IDs

Full list of state & county ids: https://www.census.gov/prod/techdoc/cbp/cbp95/st-cnty.pdf

## Read shapefiles

In [24]:
def read_shapefiles(sids):
    dfs = []
    for sid in sids:
        df = gpd.read_file(f'data_aarp_census/census/block_group_shapefiles/{sid}/tl_2018_{sid}_bg.shp')
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

shape_df = read_shapefiles([34, 36])  # NJ and NY
shape_df

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,34,013,017302,1,340130173021,Block Group 1,G5030,S,1068288,0,+40.7959633,-074.2754595,"POLYGON ((-74.28439 40.79932, -74.28271 40.802..."
1,34,013,017302,2,340130173022,Block Group 2,G5030,S,375431,0,+40.7964973,-074.2826621,"POLYGON ((-74.28704 40.79442, -74.28669 40.795..."
2,34,013,017400,1,340130174001,Block Group 1,G5030,S,3339601,3649,+40.8171740,-074.2470590,"POLYGON ((-74.26024 40.81086, -74.26008 40.811..."
3,34,013,017400,2,340130174002,Block Group 2,G5030,S,360833,0,+40.8043974,-074.2535816,"POLYGON ((-74.25795 40.80542, -74.25791 40.805..."
4,34,013,017400,3,340130174003,Block Group 3,G5030,S,1709060,18160,+40.7992993,-074.2575472,"POLYGON ((-74.26722 40.79706, -74.26713 40.797..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21778,36,047,023100,1,360470231001,Block Group 1,G5030,S,39857,0,+40.6879848,-073.9612764,"POLYGON ((-73.96284 40.68856, -73.96093 40.688..."
21779,36,047,023100,3,360470231003,Block Group 3,G5030,S,39835,0,+40.6839002,-073.9609460,"POLYGON ((-73.96211 40.68489, -73.96115 40.685..."
21780,36,047,023200,1,360470232001,Block Group 1,G5030,S,56510,0,+40.6342211,-073.9839896,"POLYGON ((-73.98538 40.63461, -73.98480 40.635..."
21781,36,047,026100,5,360470261005,Block Group 5,G5030,S,39883,0,+40.6933351,-073.9473502,"POLYGON ((-73.94891 40.69391, -73.94608 40.694..."


In [25]:
# GeoJSON(json.loads(shape_df.to_json()))

# Data wrangling

## Read all NJ and NY census median rent data

In [26]:
unique_state_county_ids = shape_df[['STATEFP', 'COUNTYFP']].drop_duplicates().to_numpy()

In [27]:
census_dfs = []
for sid, cid in unique_state_county_ids:
    print(sid, cid)
    df = get_census_data('B25058_001E', sid, cid)
    census_dfs.append(df)
census_df = pd.concat(census_dfs)

34 013
34 029
34 031
34 037
34 005
34 003
34 011
34 035
34 023
34 001
34 015
34 039
34 007
34 025
34 009
34 019
34 041
34 017
34 027
34 021
34 033
36 111
36 003
36 029
36 103
36 117
36 005
36 055
36 079
36 061
36 059
36 081
36 063
36 019
36 065
36 119
36 093
36 001
36 023
36 105
36 041
36 101
36 069
36 007
36 075
36 091
36 047
36 009
36 051
36 109
36 083
36 039
36 027
36 099
36 045
36 011
36 089
36 067
36 085
36 115
36 087
36 015
36 021
36 095
36 017
36 073
36 123
36 113
36 097
36 043
36 035
36 013
36 057
36 071
36 033
36 031
36 107
36 121
36 077
36 037
36 049
36 025
36 053


In [28]:
census_df = census_df.sort_values(['state', 'county', 'tract', 'block group']).reset_index(drop=True)

In [29]:
census_df['state']

0        34
1        34
2        34
3        34
4        34
         ..
21778    36
21779    36
21780    36
21781    36
21782    36
Name: state, Length: 21783, dtype: object

In [315]:
census_df['GEOID'] = census_df.apply(lambda x:  x['state'] + x['county'] + x['tract'] + x['block group'], axis=1)
census_df

,B25058_001E,state,county,tract,block group,GEOID
0,1042,34,001,000100,1,340010001001
1,833,34,001,000100,2,340010001002
2,894,34,001,000200,1,340010002001
3,991,34,001,000200,2,340010002002
4,1157,34,001,000200,3,340010002003
...,...,...,...,...,...,...
21778,689,36,123,150400,4,361231504004
21779,487,36,123,150500,1,361231505001
21780,535,36,123,150500,2,361231505002
21781,468,36,123,150500,3,361231505003


In [316]:
census_df.to_csv('data_aarp_census/census/medium_contract_rent_nj_ny.csv', index=False)

In [30]:
census_df = pd.read_csv('data_aarp_census/census/medium_contract_rent_nj_ny.csv', dtype={'state': str, 'county': str, 'tract': str, 'block group': str, 'GEOID': str})

In [31]:
census_df

,B25058_001E,state,county,tract,block group,GEOID
0,1042,34,001,000100,1,340010001001
1,833,34,001,000100,2,340010001002
2,894,34,001,000200,1,340010002001
3,991,34,001,000200,2,340010002002
4,1157,34,001,000200,3,340010002003
...,...,...,...,...,...,...
21778,689,36,123,150400,4,361231504004
21779,487,36,123,150500,1,361231505001
21780,535,36,123,150500,2,361231505002
21781,468,36,123,150500,3,361231505003


In [32]:
census_df.dtypes

B25058_001E     int64
state          object
county         object
tract          object
block group    object
GEOID          object
dtype: object

## Read all NJ and NY AARP livability data

### Read by recusively divide the bounding box into four quadrants if exceeding the 1000-block-group limit

First determine the overall bounding box

In [33]:
shape_df[['minx', 'miny', 'maxx', 'maxy']] = shape_df['geometry'].bounds
minx = min(shape_df['minx'])
miny = min(shape_df['miny'])
maxx = max(shape_df['maxx'])
maxy = max(shape_df['maxy'])
minx, miny, maxx, maxy

(-79.76259, 38.788657, -71.777491, 45.015865)

In [ ]:
def get_aarp_livability_recursive(minx, miny, maxx, maxy):
    df = read_aarp_livability(minx, miny, maxx, maxy)
#     print(minx, miny, maxx, maxy, len(df) if df is not None else None)
    if df is None or len(df) < 1000:  # AARP limits to return only 1000 block groups
        return df
    midx = (minx + maxx) * .5
    midy = (miny + maxy) * .5
    lower_left_df = get_aarp_livability_bound(minx, miny, midx, midy)
    lower_right_df = get_aarp_livability_bound(midx, miny, maxx, midy)
    upper_right_df = get_aarp_livability_bound(midx, midy, maxx, maxy)
    upper_left_df = get_aarp_livability_bound(minx, midy, midx, maxy)
    sub_dfs = [lower_left_df, lower_right_df, upper_right_df, upper_left_df]
    df = pd.concat([sub_df for sub_df in sub_dfs if sub_df is not None])
    df = df.reset_index(drop=True).drop_duplicates()
    return df

aarp_df = get_aarp_livability_recursive(minx, miny, maxx, maxy)
len(aarp_df)

In [2]:
aarp_df = pd.read_pickle('data_aarp_census/aarp/nj_ny_raw.pickle')
aarp_df.head()

,state_fips,total_popu,pct_africa,pct_asian,pct_hispan,pct_50plus,pct_65plus,median_inc,pct_povert,pct_disabi,...,prox_vacant,trans_freq,trans_access,trans_walk_trip,trans_delay,trans_cost,trans_fatal,trans_speed_lim,total_score,geometry
0,24,761,1,2,0,64,27,50547,2,18,...,0.483,0.0,1.0,0.46,0.0,14441,65.023,37.09,37.714286,"MULTIPOLYGON (((-78.49626 39.60436, -78.49496 ..."
1,24,1662,2,0,2,43,21,41602,13,18,...,0.214,0.0,1.0,0.48,0.0,14371,23.263,37.62,46.714286,"MULTIPOLYGON (((-78.66648 39.64913, -78.66557 ..."
2,24,1109,1,0,1,34,18,55313,8,18,...,0.236,0.0,1.0,0.47,0.0,14069,50.447,27.99,46.142857,"MULTIPOLYGON (((-78.65719 39.60715, -78.65681 ..."
3,24,1058,1,0,5,28,12,47330,10,18,...,0.283,0.0,1.0,0.46,0.0,14046,10.469,27.49,46.000000,"MULTIPOLYGON (((-78.73159 39.58253, -78.73092 ..."
4,24,945,0,0,0,42,21,31875,30,18,...,0.025,0.0,1.0,0.49,0.0,13495,7.504,39.79,45.142857,"MULTIPOLYGON (((-78.71766 39.70091, -78.71264 ..."


### Filter by NJ and NY

In [3]:
aarp_df =  aarp_df[np.isin(aarp_df.state_fips, [34, 36])]
len(aarp_df)

21783

### Build GEOID to internal point mapping

In [6]:
GEOID_TO_INTPS = {}
for i, geoid, lat, lon in shape_df[['GEOID', 'INTPTLAT', 'INTPTLON']].itertuples():
    GEOID_TO_INTPS[geoid] = Point(float(lon), float(lat))

### Use GEOID to internal point mapping to mark the AARP block group ploy with the right GEOID

In [7]:
find_count = 0

def find_block_group(polygon):
    global find_count
    if find_count % 100 == 0:
        print(f'identified {find_count} block groups')
    find_count += 1
    for geoid, intp in GEOID_TO_INTPS.items():
        if polygon.contains(intp):
            return geoid
    return None

aarp_df['GEOID'] = aarp_df['geometry'].apply(lambda x: find_block_group(x))

identified 0 block groups
identified 100 block groups
identified 200 block groups
identified 300 block groups
identified 400 block groups
identified 500 block groups
identified 600 block groups
identified 700 block groups
identified 800 block groups
identified 900 block groups
identified 1000 block groups
identified 1100 block groups
identified 1200 block groups
identified 1300 block groups
identified 1400 block groups
identified 1500 block groups
identified 1600 block groups
identified 1700 block groups
identified 1800 block groups
identified 1900 block groups
identified 2000 block groups
identified 2100 block groups
identified 2200 block groups
identified 2300 block groups
identified 2400 block groups
identified 2500 block groups
identified 2600 block groups
identified 2700 block groups
identified 2800 block groups
identified 2900 block groups
identified 3000 block groups
identified 3100 block groups
identified 3200 block groups
identified 3300 block groups
identified 3400 block grou

In [8]:
aarp_df.to_pickle('data_aarp_census/aarp/nj_ny_geoid.pickle')

### Merge AARP livability dataset with Census median rent dataset by GEOID

In [ ]:
all_df = aarp_df.merge(census_df, how='outer', on='GEOID')

In [31]:
all_df = all_df.drop(columns=['geometry', 'state_fips'])
all_df

,total_popu,pct_africa,pct_asian,pct_hispan,pct_50plus,pct_65plus,median_inc,pct_povert,pct_disabi,pct_novehi,...,trans_cost,trans_fatal,trans_speed_lim,total_score,GEOID,B25058_001E,state,county,tract,block group
0,1112.0,8.0,0.0,21.0,31.0,11.0,64250.0,0.0,12.0,5.0,...,15034.0,19.252,33.40,43.857143,340010107001,-666666666.0,34,001,010700,1
1,1203.0,3.0,0.0,2.0,37.0,11.0,69375.0,4.0,12.0,1.0,...,15223.0,24.897,41.57,44.714286,340010112011,917.0,34,001,011201,1
2,2303.0,3.0,1.0,16.0,42.0,13.0,72870.0,14.0,12.0,3.0,...,14611.0,8.331,33.17,47.000000,340010112021,1201.0,34,001,011202,1
3,1534.0,0.0,0.0,1.0,60.0,41.0,39196.0,10.0,12.0,6.0,...,14096.0,8.375,29.43,46.000000,340010112022,1417.0,34,001,011202,2
4,1295.0,2.0,0.0,4.0,34.0,13.0,75750.0,2.0,12.0,6.0,...,15039.0,23.952,28.58,44.714286,340010112023,-666666666.0,34,001,011202,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,340312460036,-666666666.0,34,031,246003,6
21788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,360470702031,-666666666.0,36,047,070203,1
21789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,360510302021,613.0,36,051,030202,1
21790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,360811008012,1690.0,36,081,100801,2


In [32]:
all_df.to_pickle('data_aarp_census/nj_ny_merged_with_census.pickle')

In [35]:
all_df[(all_df['B25058_001E'] > 0) & (all_df['total_score'] > 0)]

,total_popu,pct_africa,pct_asian,pct_hispan,pct_50plus,pct_65plus,median_inc,pct_povert,pct_disabi,pct_novehi,...,trans_cost,trans_fatal,trans_speed_lim,total_score,GEOID,B25058_001E,state,county,tract,block group
1,1203.0,3.0,0.0,2.0,37.0,11.0,69375.0,4.0,12.0,1.0,...,15223.0,24.897,41.57,44.714286,340010112011,917.0,34,001,011201,1
2,2303.0,3.0,1.0,16.0,42.0,13.0,72870.0,14.0,12.0,3.0,...,14611.0,8.331,33.17,47.000000,340010112021,1201.0,34,001,011202,1
3,1534.0,0.0,0.0,1.0,60.0,41.0,39196.0,10.0,12.0,6.0,...,14096.0,8.375,29.43,46.000000,340010112022,1417.0,34,001,011202,2
5,2466.0,33.0,0.0,22.0,30.0,19.0,44318.0,23.0,12.0,6.0,...,15141.0,16.489,30.35,45.571429,340010112024,843.0,34,001,011202,4
6,1278.0,11.0,2.0,47.0,23.0,4.0,69938.0,1.0,12.0,0.0,...,14081.0,9.852,28.31,47.857143,340010113001,1151.0,34,001,011300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21778,1645.0,0.0,0.0,6.0,32.0,20.0,49185.0,9.0,13.0,4.0,...,15705.0,14.666,31.60,42.428571,360750203022,653.0,36,075,020302,2
21779,863.0,0.0,1.0,2.0,30.0,8.0,38333.0,20.0,13.0,0.0,...,15214.0,15.275,32.45,46.428571,360750205003,475.0,36,075,020500,3
21780,1586.0,1.0,0.0,2.0,32.0,11.0,49583.0,14.0,13.0,2.0,...,15333.0,16.890,29.71,44.428571,360750215011,543.0,36,075,021501,1
21781,980.0,3.0,0.0,1.0,37.0,11.0,32813.0,36.0,13.0,0.0,...,14359.0,3.355,25.00,40.571429,360750215021,565.0,36,075,021502,1


# Appendix

#### Read by county --- NOT USED as it does not handle the 1000-block-group limit